# Web scraping - kommande v75-omgång

In [2]:
#!apt install chromium-chromedriver
!pip install selenium
!pip install tabulate

In [3]:
import pandas as  pd 
import numpy as np
import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(message)s',level=logging.INFO)
import sys
sys.path.append('C:\\Users/peter/Google Drive/Colab Notebooks/Småprojekt/')
import fixa_features as ff
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
import sklearn

print("sklearn version", sklearn.__version__)

sklearn version 0.21.3


# Ladda v75-sida med 7 avdelningar/lopp 

In [4]:
def V75_sida(V75page):
    # run firefox webdriver from executable path of your choice
    logging.warning('STARTING V75_sida')
    print('starta V75_sida')
    driver = webdriver.Chrome(executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe')
    #####
    driver.implicitly_wait(10) # seconds

    # get web pages
    data = []
    driver.get(V75page+'avd1')
    but = driver.find_element_by_id('onetrust-accept-btn-handler')
    but.click()
    driver.fullscreen_window()
  
    anp = driver.find_elements_by_css_selector('div.active-game-type-v75 div.startlist-header.game-background-pattern--long button:nth-child(3)')
    anp[0].click()

    cl = driver.find_elements_by_css_selector('div.modal-container  div.modal__body  button.css-tqseha-Button-styles--root-Button--Button')
    cl[0].click()    # Rensa


    cl = driver.find_elements_by_css_selector('ul > li:nth-child(10)')
    cl[1].click()       #Distans och spår

    #cl = driver.find_elements_by_css_selector('ul > li:nth-child(8)')
    #cl[1].click()       #vodds

    #cl = driver.find_elements_by_css_selector('ul > li:nth-child(9)')
    #cl[1].click()       #vodds

    #cl = driver.find_elements_by_css_selector('div.modal-container  div.modal__body #button.css-1fh4n7y-Button-styles--root-PrimaryButton-styles--root-StartlistDisplayOptionsDialog-styles--saveButton-PrimaryButton--PrimaryButton-#StartlistDisplayOptionsDialog-styles--saveButton')
 #   cl[0].click()       #save

    print(V75page)
    print('set full screen om inte gjort och anpassa')

    return driver


In [5]:
#anp = driver.find_elements_by_css_selector('div.active-game-type-v75 div.startlist-header.game-background-pattern--long button:nth-child(3)')
#print(len(anp))
#anp[0].click()


# Fixa features

In [6]:
def fixa_features(df):
  print('fix features')
  #df['spår'] = df.spår.astype(float)
  #
  #df.loc[df.plac == 'd','plac']= 14
  #df.loc[df.plac == '0','plac']= 13
  #df.loc[df.plac == '0:a','plac']= 13
  #df.loc[df.plac == '1:a','plac']= '1'
  #df['plac'] = df.loc[:,'plac'].astype('float')
  #df.h1_plac.value_counts(dropna=False)

  df.loc[df.start == 'AUTOSTART','start']= 0
  df.loc[df.start == 'VOLTSTART','start']= 1
  df['start'] = df.start.astype('int64')

  df.vodds = df['vodds'].str.replace(',','.').astype('float')

  ## podds: räkna om 'x.xx - y.yy' till ett medelvärde
  def new_podds(p):
    x=p.replace(',' , '.')
    sp=x.split('-')

    x=sp[0]
    if len(sp) == 2:
      x = (float(sp[0])+float(sp[1]))/2
    
    return x

  # print(df['podds'].str.contains('-').sum())
  df['podds'] = df['podds'].apply(lambda x: new_podds(x))
  df['podds'] = df.podds.astype('float')
  ##

  #### hx_hist['plac'] #### 
   
  df.h1_plac = df['h1_plac'].str.replace(',','.')
  df.h2_plac = df['h2_plac'].str.replace(',','.')
  df.h3_plac = df['h3_plac'].str.replace(',','.')
  df.h4_plac = df['h4_plac'].str.replace(',','.')
  df.h5_plac = df['h5_plac'].str.replace(',','.')
  
  # byt '' till 13
  df.loc[df.h1_plac == '','h1_plac']= 13
  df.loc[df.h2_plac == '','h2_plac']= 13
  df.loc[df.h3_plac == '','h3_plac']= 13
  df.loc[df.h4_plac == '','h4_plac']= 13
  df.loc[df.h5_plac == '','h5_plac']= 13

  # byt '0' till 13
  df.loc[df.h1_plac == '0','h1_plac']= 13
  df.loc[df.h2_plac == '0','h2_plac']= 13
  df.loc[df.h3_plac == '0','h3_plac']= 13
  df.loc[df.h4_plac == '0','h4_plac']= 13
  df.loc[df.h5_plac == '0','h5_plac']= 13

  # byt 'd' till 14
  df.loc[df.h1_plac == 'd','h1_plac']= 14
  df.loc[df.h2_plac == 'd','h2_plac']= 14
  df.loc[df.h3_plac == 'd','h3_plac']= 14
  df.loc[df.h4_plac == 'd','h4_plac']= 14
  df.loc[df.h5_plac == 'd','h5_plac']= 14

  # ett 'r' för mycket
  df.loc[df.h1_plac == '0r','h1_plac']= 13
  df.loc[df.h2_plac == '0r','h2_plac']= 13
  df.loc[df.h3_plac == '0r','h3_plac']= 13
  df.loc[df.h4_plac == '0r','h4_plac']= 13
  df.loc[df.h5_plac == '0r','h5_plac']= 13

  df.loc[df.h1_plac == '2r','h1_plac']= 2
  df.loc[df.h2_plac == '2r','h2_plac']= 2
  df.loc[df.h3_plac == '2r','h3_plac']= 2
  df.loc[df.h4_plac == '2r','h4_plac']= 2
  df.loc[df.h5_plac == '2r','h5_plac']= 2

  df.loc[df.h1_plac == '3r','h1_plac']= 3
  df.loc[df.h2_plac == '3r','h2_plac']= 3
  df.loc[df.h3_plac == '3r','h3_plac']= 3
  df.loc[df.h4_plac == '3r','h4_plac']= 3
  df.loc[df.h5_plac == '3r','h5_plac']= 3

  df.loc[df.h1_plac == '4r','h1_plac']= 4
  df.loc[df.h2_plac == '4r','h2_plac']= 4
  df.loc[df.h3_plac == '4r','h3_plac']= 4
  df.loc[df.h4_plac == '4r','h4_plac']= 4
  df.loc[df.h5_plac == '4r','h5_plac']= 4

  # byt 'k' till NaN
  df.loc[df.h1_plac == 'k', 'h1_dist'] = df.loc[df.h1_plac == 'k','h1_dist'].apply(lambda x: np.nan) 
  df.loc[df.h1_plac == 'k', 'h1_spår'] = df.loc[df.h1_plac == 'k','h1_spår'].apply(lambda x: np.nan) 
  df.loc[df.h1_plac == 'k', 'h1_odds'] = df.loc[df.h1_plac == 'k','h1_odds'].apply(lambda x: np.nan) 
  df.loc[df.h1_plac == 'k', 'h1_plac'] = df.loc[df.h1_plac == 'k','h1_plac'].apply(lambda x: np.nan) 

  df.loc[df.h2_plac == 'k', 'h2_dist'] = df.loc[df.h2_plac == 'k','h2_dist'].apply(lambda x: np.nan) 
  df.loc[df.h2_plac == 'k', 'h2_spår'] = df.loc[df.h2_plac == 'k','h2_spår'].apply(lambda x: np.nan) 
  df.loc[df.h2_plac == 'k', 'h2_odds'] = df.loc[df.h2_plac == 'k','h2_odds'].apply(lambda x: np.nan) 
  df.loc[df.h2_plac == 'k', 'h2_plac'] = df.loc[df.h2_plac == 'k','h2_plac'].apply(lambda x: np.nan) 

  df.loc[df.h3_plac == 'k', 'h3_dist'] = df.loc[df.h3_plac == 'k','h3_dist'].apply(lambda x: np.nan) 
  df.loc[df.h3_plac == 'k', 'h3_spår'] = df.loc[df.h3_plac == 'k','h3_spår'].apply(lambda x: np.nan) 
  df.loc[df.h3_plac == 'k', 'h3_odds'] = df.loc[df.h3_plac == 'k','h3_odds'].apply(lambda x: np.nan) 
  df.loc[df.h3_plac == 'k', 'h3_plac'] = df.loc[df.h3_plac == 'k','h3_plac'].apply(lambda x: np.nan) 

  df.loc[df.h4_plac == 'k', 'h4_dist'] = df.loc[df.h4_plac == 'k','h4_dist'].apply(lambda x: np.nan) 
  df.loc[df.h4_plac == 'k', 'h4_spår'] = df.loc[df.h4_plac == 'k','h4_spår'].apply(lambda x: np.nan) 
  df.loc[df.h4_plac == 'k', 'h4_odds'] = df.loc[df.h4_plac == 'k','h4_odds'].apply(lambda x: np.nan) 
  df.loc[df.h4_plac == 'k', 'h4_plac'] = df.loc[df.h4_plac == 'k','h4_plac'].apply(lambda x: np.nan) 

  df.loc[df.h5_plac == 'k', 'h5_dist'] = df.loc[df.h5_plac == 'k','h5_dist'].apply(lambda x: np.nan) 
  df.loc[df.h5_plac == 'k', 'h5_spår'] = df.loc[df.h5_plac == 'k','h5_spår'].apply(lambda x: np.nan) 
  df.loc[df.h5_plac == 'k', 'h5_odds'] = df.loc[df.h5_plac == 'k','h5_odds'].apply(lambda x: np.nan) 
  df.loc[df.h5_plac == 'k', 'h5_plac'] = df.loc[df.h5_plac == 'k','h5_plac'].apply(lambda x: np.nan) 

  # byt 'p' till NaN
  df.loc[df.h2_plac == 'p', 'h2_dist'] = df.loc[df.h2_plac == 'p','h2_dist'].apply(lambda x: np.nan)
  df.loc[df.h2_plac == 'p', 'h2_spår'] = df.loc[df.h2_plac == 'p','h2_spår'].apply(lambda x: np.nan)
  df.loc[df.h2_plac == 'p', 'h2_odds'] = df.loc[df.h2_plac == 'p','h2_odds'].apply(lambda x: np.nan)
  df.loc[df.h2_plac == 'p', 'h2_plac'] = df.loc[df.h2_plac == 'p','h2_plac'].apply(lambda x: np.nan)

  df.loc[df.h5_plac == 'p']
  df.loc[df.h5_plac == 'p', 'h5_dist'] = df.loc[df.h5_plac == 'p','h5_dist'].apply(lambda x: np.nan)
  df.loc[df.h5_plac == 'p', 'h5_spår'] = df.loc[df.h5_plac == 'p','h5_spår'].apply(lambda x: np.nan)
  df.loc[df.h5_plac == 'p', 'h5_odds'] = df.loc[df.h5_plac == 'p','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_plac == 'p', 'h5_plac'] = df.loc[df.h5_plac == 'p','h5_plac'].apply(lambda x: np.nan)

  # gör till int32
  df['h1_plac'] = df.h1_plac.astype('float')
  df['h2_plac'] = df.h2_plac.astype('float')
  df['h3_plac'] = df.h3_plac.astype('float')
  df['h4_plac'] = df.h4_plac.astype('float')
  df['h5_plac'] = df.h5_plac.astype('float')

  #### hx_odds ####

  #
  df.h1_odds = df['h1_odds'].str.replace(',','.')
  df.loc[df.h1_odds == '', 'h1_odds'] = df.loc[df.h1_odds == '','h1_odds'].apply(lambda x: np.nan)  
  df.loc[df.h1_odds == '-', 'h1_odds'] = df.loc[df.h1_odds == '-','h1_odds'].apply(lambda x: np.nan)
  df.loc[df.h1_odds=='GDK','h1_odds'] = df.loc[df.h1_odds == 'GDK','h1_odds'].apply(lambda x: np.nan)
  df['h1_odds'] = df.h1_odds.astype('float')

  #h2
  df.h2_odds = df['h2_odds'].str.replace(',','.')
  df.loc[df.h2_odds == '', 'h2_odds'] = df.loc[df.h2_odds == '','h2_odds'].apply(lambda x: np.nan)  
  df.loc[df.h2_odds == '-', 'h2_odds'] = df.loc[df.h2_odds == '-','h2_odds'].apply(lambda x: np.nan)
  df.loc[df.h2_odds=='GDK','h2_odds'] = df.loc[df.h2_odds == 'GDK','h2_odds'].apply(lambda x: np.nan)
  df.loc[df.h2_odds=='gdk','h2_odds'] = df.loc[df.h2_odds == 'gdk','h2_odds'].apply(lambda x: np.nan)
  df['h2_odds'] = df.h2_odds.astype('float')

  #h3
  df.h3_odds = df['h3_odds'].str.replace(',','.')
  df.loc[df.h3_odds == '', 'h3_odds'] = df.loc[df.h3_odds == '','h3_odds'].apply(lambda x: np.nan)  
  df.loc[df.h3_odds == '-', 'h3_odds'] = df.loc[df.h3_odds == '-','h3_odds'].apply(lambda x: np.nan)
  df.loc[df.h3_odds=='GDK','h3_odds'] = df.loc[df.h3_odds == 'GDK','h3_odds'].apply(lambda x: np.nan)
  df.loc[df.h3_odds=='gdk','h3_odds'] = df.loc[df.h3_odds == 'gdk','h3_odds'].apply(lambda x: np.nan)
  df.loc[df.h3_odds=='b-lp','h3_odds'] = df.loc[df.h3_odds == 'b-lp','h3_odds'].apply(lambda x: np.nan)
  df.loc[df.h3_odds=='ejg','h3_odds'] = df.loc[df.h3_odds == 'ejg','h3_odds'].apply(lambda x: np.nan)
  df.loc[df.h3_odds=='ut','h3_odds'] = df.loc[df.h3_odds == 'ut','h3_odds'].apply(lambda x: np.nan)
  df['h3_odds'] = df.h3_odds.astype('float')

  #h4
  df.h4_odds = df['h4_odds'].str.replace(',','.')
  df.loc[df.h4_odds == '', 'h4_odds'] = df.loc[df.h4_odds == '','h4_odds'].apply(lambda x: np.nan)  
  df.loc[df.h4_odds == '-', 'h4_odds'] = df.loc[df.h4_odds == '-','h4_odds'].apply(lambda x: np.nan)
  df.loc[df.h4_odds=='GDK','h4_odds'] = df.loc[df.h4_odds == 'GDK','h4_odds'].apply(lambda x: np.nan)
  df.loc[df.h4_odds=='gdk','h4_odds'] = df.loc[df.h4_odds == 'gdk','h4_odds'].apply(lambda x: np.nan)
  df.loc[df.h4_odds=='b-lp','h4_odds'] = df.loc[df.h4_odds == 'b-lp','h4_odds'].apply(lambda x: np.nan)
  df.loc[df.h4_odds=='ejg','h4_odds'] = df.loc[df.h4_odds == 'ejg','h4_odds'].apply(lambda x: np.nan)
  df.loc[df.h4_odds=='ut','h4_odds'] = df.loc[df.h4_odds == 'ut','h4_odds'].apply(lambda x: np.nan)
  df['h4_odds'] = df.h4_odds.astype('float')

  #h5
  df.h5_odds = df['h5_odds'].str.replace(',','.')
  df.loc[df.h5_odds == '', 'h5_odds'] = df.loc[df.h5_odds == '','h5_odds'].apply(lambda x: np.nan)  
  df.loc[df.h5_odds == '-', 'h5_odds'] = df.loc[df.h5_odds == '-','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='GDK','h5_odds'] = df.loc[df.h5_odds == 'GDK','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='gdk','h5_odds'] = df.loc[df.h5_odds == 'gdk','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='b-lp','h5_odds'] = df.loc[df.h5_odds == 'b-lp','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='B-lp','h5_odds'] = df.loc[df.h5_odds == 'B-lp','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='ejg','h5_odds'] = df.loc[df.h5_odds == 'ejg','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='ut','h5_odds'] = df.loc[df.h5_odds == 'ut','h5_odds'].apply(lambda x: np.nan)
  df.loc[df.h5_odds=='P','h5_odds'] = df.loc[df.h5_odds == 'P','h5_odds'].apply(lambda x: np.nan)
  #df.loc[df.h5_odds=='B-lp']
  df['h5_odds'] = df.h5_odds.astype('float')

  df.dropna(subset=['spår'],inplace=True)
  df.dropna(subset=['h1_plac','h1_odds','h2_plac','h2_odds','h3_plac','h3_odds'],inplace=True)
  df.dropna(inplace=True)

  df['dist'] = df.dist.astype('float')
  return df

In [7]:
def getSpår(startnr, avd, i):
    logging.warning('startar getSpår')
    #pth='div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(i)+') > td.postPositionAndDistance-col'
    try:    
        spår = startnr.find_element_by_css_selector('.postPositionAndDistance-col')

        txt = spår.text.split()
        spr = None
        dis = None

        if len(txt) > 0:
            dis = txt[0]
    
        if len(txt) == 3:
            spr = txt[2]
    except:
        logging.warning('dist/spår hittades ej')
        spr='ej'
        dis='ej'
    return spr,dis



In [8]:
def getVodds(startnr, avd, häst):
    logging.warning('startar getVodds')

    #pth = '#main > div.active-game-type-v75 > div:nth-child(2) > div > div > div > div > div > div:nth-child(7) > div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(häst)+') > td.vOdds-col'
          
    vo = startnr.find_element_by_css_selector('.vOdds-col')

    return vo.text    


In [9]:
def getPodds(startnr, avd, häst):
    logging.warning('startar getPodds')
    #pth = '#main > div.active-game-type-v75 > div:nth-child(2) > div > div > div > div > div > div:nth-child(7) > div:nth-child('+str(avd)+') > div > table > tbody > tr:nth-child('+str(häst)+') > td.pOdds-col'
         
    po = startnr.find_element_by_css_selector('.pOdds-col')
    
    return po.text    


In [10]:
def getHist(avd, häst):
    logging.warning('startar getHist')
    history=[]
    try:
        for child in range(1,6):
            dist = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.distance-col'
            plac = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.place-col'
            spår = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.position-col'
            odds = 'table > tbody > tr:nth-child(' +str(child)+ ') > td.odds-col'
            
            plac = driver.find_element_by_css_selector(plac)
            logging.warning('plac '+plac.text)
            dist = driver.find_element_by_css_selector(dist)
            logging.warning('dist '+dist.text)
            spår = driver.find_element_by_css_selector(spår)
            logging.warning('spår '+spår.text)
            odds = driver.find_element_by_css_selector(odds)
            logging.warning('odds '+odds.text)
            history.extend([plac.text, dist.text, spår.text, odds.text])
    except:
        logging.warning("history except")
        return history
    
    logging.warning('History = '+ str(history))
    return history

# Läs in avdelning för avdelning och skrapa
öppna varje lopp och gå igenom all hästarna inklusive deras historik

In [11]:
def skrapa(avdelningar, V75page):
    driver.implicitly_wait(10) # seconds
    logging.warning('STARTING skrapa')
    print('starta skrapa')
    
    dat = V75page.split('spel/')[1][0:10]  #gräv ut datum
    komplett=[]
    for avd in avdelningar:
        logging.warning('avd '+ str(avd))

        # open avd
        driver.get(V75page+'avd'+str(avd))
        d=driver.find_element_by_css_selector('div > div.cell-wrapper > div.race-combined-info > div.slanted.race-distance-start-method > span')    
        
        split1 = d.text.split()[0][0:4]   # distans (använd hästens distans ist om den finns)
        split2 = d.text.split()[1]        # start

        css='#main .game-view table[data-test-id="startlist-race-'+str(avd)+'"] tbody tr[data-test-id="startlist-row"]'
        logging.warning(css)
        gt = driver.find_elements_by_css_selector(css) 
        antal = len(gt)
        print('antal',antal)
        häst = 0
        for startnr in gt:
            häst = häst+1
            #pth ='//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+str(avd)+']/div/table/tbody/tr['+str(häst)+']/td[1]/span/div/span'
            #pth = '//*[@id="main"]/div[3]/div[2]/div/div/div/div/div/div[7]/div['+str(avd)+']//*[@class]'
            #logging.warning(pth)
            try:
                #print('startnr.text',startnr.text)
                nm = startnr.find_element_by_css_selector('.name-container')
                namn = nm.text
                print(avd, häst, nm.text)
            except:
                logging.warning('avd '+str(avd)+' Startnr: '+str(häst)+'  hittades inte')
                break    

            spår, dist = getSpår(startnr, avd, häst)
            
            if spår.lower()=='ej':
                logging.warning('avd: '+str(avd)+' startnr: '+str(häst)+' struken')
                continue
            else:   
                if dist==None:
                    dist = split1  # använd loppdistans om det saknas hos hästen
                #namn = gt.text
                vodds = getVodds(startnr, avd,häst)
                podds = getPodds(startnr, avd,häst)
                if vodds.lower() == 'ej' or podds.lower() == 'ej':
                    logging.warning('avd: '+str(avd)+' startnr: '+str(häst)+' struken')
                    continue    

                logging.warning('avd: '+str(avd)+' Startnr: '+str(häst)+' '+namn)  

                start = split2

                horse = [dat, avd, namn, dist, start, spår, vodds, podds]
                
                startnr.click()  # open for history
                
                #gh=getHist(avd,häst)
                horse.extend(getHist(avd,häst))
                komplett.append(horse)
                #print('history:',history)
                startnr.click()   # close

    return komplett

    

# Kör fram resultat

In [12]:
def resultat(pool, model_name, modelpath):
    from sklearn import model_selection
    #from sklearn.linear_model import LogisticRegression
    import pickle
    
    if modelpath:
        # load the model from disk
        loaded_model = pickle.load(open(modelpath+model_name, 'rb'))

    result = loaded_model.predict(pool)
    return result

In [13]:
def resultat_proba(pool, model_name, modelpath):  
    from sklearn import model_selection
    import pickle
    
    if modellpath:
        # load the model from disk
        loaded_model = pickle.load(open(modellpath+model_name, 'rb'))

    result = loaded_model.predict_proba(pool)

    return result[:,1]

# spara lopp data

In [14]:
def spara(df,path, omgsav, avd):
    stravd = ''.join(str(x) for x in avd)
    filnamn=path+omgsav+stravd+'test.csv'
    df.to_csv(filnamn, index=False)

# Main - init

In [15]:
omg = '2020-05-09/V75/aby'
omg_sav = omg.replace('/V75/','-')

V75url = 'https://www.atg.se/spel/'+omg
driver=V75_sida(V75url)
#2020-05-09/V75/aby
#2020-04-18/V75/umaker  
#2020-05-02/V75/orebro  
#2020-04-25/V75/aby    

starta V75_sida
https://www.atg.se/spel/2020-05-09/V75/aby
set full screen om inte gjort och anpassa


# Main - web scraping plus predict och spara

In [23]:
avd = [7]  ## range(1,8) för alla avdelningar, [2, 4] för 2 och 4

if True:
    #komplett=skrapa(avd, V75url)  
    
    df=pd.DataFrame(komplett)
    df.columns=['datum','avd','häst', 'dist', 'start', 'spår', 'vodds', 'podds' , 
    'h1_plac', 'h1_dist', 'h1_spår', 'h1_odds',
    'h2_plac', 'h2_dist', 'h2_spår', 'h2_odds',
    'h3_plac', 'h3_dist', 'h3_spår', 'h3_odds',
    'h4_plac', 'h4_dist', 'h4_spår', 'h4_odds',
    'h5_plac', 'h5_dist', 'h5_spår', 'h5_odds']

    df = ff.fix_features(df, False)
    
if True:  # kör resultat
    
    from catboost import Pool
    poolUtan = Pool(
        cat_features=['start', 'häst'],
        data=df.drop(['vodds','datum','avd'],axis=1)
    )
    poolMed = Pool(
        cat_features=['start', 'häst'],
        data=df.drop(['datum','avd'],axis=1)
    )
    lim={   'regr': {'pool': poolUtan,'thresh': -0.9, 'odds_fr': 1.0, 'odds_to': 8.4},
            'class':{'pool': poolUtan,'thresh': 1.2, 'odds_fr': 1.6, 'odds_to': 20.1, 'proba_lim':0}, 
        }
    
    #LÄS IN MODELLER
    #modellpath = 'C:\\Users/peter/Documents/MyProjects/PyProj/Trav/Modeller/'
    modellpath =  'C:\\Users/peter/Google Drive/Colab Notebooks/Småprojekt/'
    
    res = resultat(lim['regr']['pool'], 'CatB_regr7.model', modellpath)     
    #res_proba = resultat_proba(poolUtan, 'CatB_class8.model', modellpath)     
    res_proba = resultat_proba(df.drop(['häst','avd','datum'],axis=1), 'NN_class2.model', modellpath)     


    #BESTÄM REGR RESULTAT
    df['regr_res'] = res
    df['regr_vodds']=df.vodds >= lim['regr']['odds_fr'] 
    df['regr_vodds']=df.regr_vodds * (df.vodds <= lim['regr']['odds_to'] ) 
    df['regr_satsa']=df.regr_res >= lim['regr']['thresh']
    df['regr_satsa']=df.regr_satsa * df.regr_vodds

    #BESTÄM CLASS RESULTAT
    df['proba_res'] = res_proba
    df['proba_vodds']=df.vodds >= lim['class']['odds_fr'] 
    df['proba_vodds']=df.proba_vodds * (df.vodds <= lim['class']['odds_to'] ) 
    df['proba_förv'] = res_proba * df.vodds
    df['proba_satsa']=df.proba_förv >= lim['class']['thresh']
    df['proba_satsa']=df.proba_satsa * (df.proba_res >= lim['class']['proba_lim'] )
    df['proba_satsa']=df.proba_satsa * df.proba_vodds

    #SKRIV UT RESULTAT
    mycolumns=pd.Series(df.columns[-1:]).append(pd.Series(df.columns[:-1]))
    df=df[mycolumns]  # flytta 'satsa' till först bland kolumner
    lopp_path='C:\\Users/peter/Documents/MyProjects/PyProj/Trav/'
    spara(df,lopp_path,omg_sav,avd)
    print("\nATT SPELA PÅ")
    print(df.loc[(df.regr_satsa==True) | (df.proba_satsa==True) ,['avd','häst','regr_satsa', 'proba_satsa']])
    print()
    
    df[['avd','häst','vodds','regr_res','regr_vodds','proba_res','proba_vodds',  'regr_satsa', 'proba_satsa']].head(60)


plac finns inte i DataFrame


ValueError: could not convert string to float: 

In [ ]:
df[['avd','häst','vodds','regr_res','regr_vodds','proba_res','proba_vodds',  'regr_satsa', 'proba_satsa']].head(60)

In [ ]:
df[['avd','häst','vodds','regr_res','regr_vodds','proba_res','proba_vodds',  'regr_satsa', 'proba_satsa']].tail(25)

In [ ]:
df.drop(['häst','avd','datum'],axis=1).info()

In [ ]:
driver.quit()

In [ ]:
df.loc[df.h1_odds == '', 'h1_odds'] = df.loc[df.h1_odds == '','h1_odds'].apply(lambda x: np.nan)
df.h1_odds.astype('float')

In [ ]:
import fixa_features as fff
fff.fix_features(df,True)

In [ ]:
lues

In [20]:
import numpy as np

In [22]:
np.__version__

'1.16.5'